In [ ]:
# %pip install -r requirements.txt
# %pip freeze > requirements.txt
# %pip install -U langchain jinja2

In [ ]:
from langchain.agents import Tool
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferMemory 
from jinja2 import Template
# from langchain.prompts import Jinja2PromptTemplate
import openai

from dotenv import load_dotenv
import json
import os
import json
import random

In [108]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
serpapi_api_key = os.getenv("SERPAPI_API_KEY")

In [109]:
JSON_FILE_PATH = "data\\project_summary.json"
RCMD_PROMPT_PATH = "cat-feat_recommend.md"

In [124]:
# Read data
with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# Read prompt
with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
    RCMD_PROMPT = f.read()

In [125]:
json_data

[{'project_info_id': 1,
  'category': '웹 서비스',
  'core_features': ['ERD 자동생성', 'API 명세서 자동생성', '프로젝트 정보 자동생성'],
  'created_at': '2025-06-18T00:39:46.947095',
  'problem_solving': {'solutionIdea': 'AI가 자동으로 설계 문서를 생성하여 개발 과정을 단순화',
   'currentProblem': '초보 개발자들이 복잡한 설계 과정을 이해하고 진행하는 데 어려움이 있음',
   'expectedBenefits': ['빠른 개발 착수', '효율적인 팀원 간 커뮤니케이션', '체계적인 프로젝트 경험']},
  'target_users': ['프로젝트 경험이 적은 초보 개발자'],
  'technology_stack': ['react',
   'typescript',
   'springboot',
   'AWS',
   'git action',
   'vercel'],
  'title': '프로젝트 관리 웹 서비스',
  'updated_at': '2025-06-18T00:39:46.947102',
  'workspace_id': 1},
 {'project_info_id': 11,
  'category': '학습 관리',
  'core_features': ['체크리스트 형태의 과제 관리', '퀴즈를 통한 반복 학습'],
  'created_at': '2025-06-19T08:24:29.404944',
  'problem_solving': {'solutionIdea': '스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있는 기능 제공',
   'currentProblem': '학생들이 학습 동기를 잃고 효과적으로 학습하지 못하는 문제',
   'expectedBenefits': ['학습 효과 극대화', '사용자 간 소통 강화', '학습 동기 부여']},
  'target_users': ['학습

In [126]:
print(RCMD_PROMPT)

당신은 신입/초보 개발자 팀을 이끄는 시니어 테크 리드입니다.  
다음은 한 프로젝트에 대한 주요 정보입니다.  
이 정보를 바탕으로 해당 프로젝트를 성공적으로 구현하기 위한 작업 구조를 작성해야 합니다.

---

- 프로젝트 정보 JSON:
{{ input }}

작업 조건:
1. Category: 패키지 수준의 상위 기능 그룹, 5개 이상  
2. Feature: 각 Category 아래 기능 모듈, 5개 이상  
3. Action: 각 Feature에 대응하는 실제 구현 단위, 3개 이상  
4. importance: 각 Actions의 기능 중요도 (1~5 사이 정수)  
5. 출력은 반드시 JSON 형식의 문자열만 반환 (설명 X, 주석 X)

예시 포맷:
{
  "workspace_id": "워크스페이스 ID",
  "recommendedCategories": [
    {
      "name": "카테고리 이름",
      "features": [
        {
          "name": "기능 이름",
          "actions": [
            { "name": "작업 이름", "importance": 정수 }
          ]
        }
      ]
    }
  ]
}



In [127]:
# project ID 추출
def extract_values_by_key(obj, key):
    values = []

    if isinstance(obj, dict):
        for k, v in obj.items():
            if k == key:
                values.append(v)
            values.extend(extract_values_by_key(v, key))
    elif isinstance(obj, list):
        for item in obj:
            values.extend(extract_values_by_key(item, key))

    return values

# Read target file
with open(JSON_FILE_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

target_key = "project_info_id"
proj_id_list = extract_values_by_key(data, target_key)

print(proj_id_list)

[1, 11, 4, 12, 6, 16, 15]


In [128]:
project_id = int(random.choice(proj_id_list))

def extract_project_sum(input_json, id):
    # peject_id로 프로젝트 개요 읽어오기
    for item in input_json:
        if item.get("project_info_id") == id:
            return item
    return None

# peject_id로 읽어온 프로젝트 개요
project_sum = extract_project_sum(data, project_id)
# project_sum  # test
type(project_sum)  # dict

if project_sum:
    print(json.dumps(project_sum, ensure_ascii=False, indent=2))  # <class 'str'>
else:
    print("** 해당 project_id를 찾을 수 없습니다. **")

{
  "project_info_id": 11,
  "category": "학습 관리",
  "core_features": [
    "체크리스트 형태의 과제 관리",
    "퀴즈를 통한 반복 학습"
  ],
  "created_at": "2025-06-19T08:24:29.404944",
  "problem_solving": {
    "solutionIdea": "스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있는 기능 제공",
    "currentProblem": "학생들이 학습 동기를 잃고 효과적으로 학습하지 못하는 문제",
    "expectedBenefits": [
      "학습 효과 극대화",
      "사용자 간 소통 강화",
      "학습 동기 부여"
    ]
  },
  "target_users": [
    "학습 동기를 얻고 싶은 학생"
  ],
  "technology_stack": [
    "javaScript",
    "java"
  ],
  "title": "스터디 그룹 운영지원 플랫폼",
  "updated_at": "2025-06-19T08:24:29.404954",
  "workspace_id": 17
}


In [129]:
project_id = int(random.choice(proj_id_list))

def extract_project_summary(input_json, id):
    # peject_id로 프로젝트 개요 중 지정 컬럼 읽어오기
    for item in input_json:
        if item.get("project_info_id") == id:
            return {
                "category": item["category"],
                "core_features": item["core_features"],
                "problem_solving": {
                    "solutionIdea": item["problem_solving"]["solutionIdea"],
                    "currentProblem": item["problem_solving"]["currentProblem"],
                    "expectedBenefits": item["problem_solving"]["expectedBenefits"]
                },
                "target_users": item["target_users"],
                "technology_stack": item["technology_stack"],
                "title": item["title"],
                "workspace_id": item["workspace_id"]
            }
        
    return None

# peject_id로 읽어온 프로젝트 개요의 지정 컬럼
project_summary = extract_project_summary(data, project_id)
# project_summary  # test
type(project_summary)  # dict

if project_summary:
    print(json.dumps(project_summary, ensure_ascii=False, indent=2))  # <class 'str'>
else:
    print("** 해당 project_id를 찾을 수 없습니다. **")

{
  "category": "웹 서비스",
  "core_features": [
    "ERD 자동생성",
    "API 명세서 자동생성",
    "프로젝트 정보 자동생성"
  ],
  "problem_solving": {
    "solutionIdea": "AI가 자동으로 설계 문서를 생성하여 개발 과정을 단순화",
    "currentProblem": "초보 개발자들이 복잡한 설계 과정을 이해하고 진행하는 데 어려움이 있음",
    "expectedBenefits": [
      "빠른 개발 착수",
      "효율적인 팀원 간 커뮤니케이션",
      "체계적인 프로젝트 경험"
    ]
  },
  "target_users": [
    "프로젝트 경험이 적은 초보 개발자"
  ],
  "technology_stack": [
    "react",
    "typescript",
    "springboot",
    "AWS",
    "git action",
    "vercel"
  ],
  "title": "프로젝트 관리 웹 서비스",
  "workspace_id": 1
}


In [130]:
# test
project_summary
# type(project_summary)  # dict

{'category': '웹 서비스',
 'core_features': ['ERD 자동생성', 'API 명세서 자동생성', '프로젝트 정보 자동생성'],
 'problem_solving': {'solutionIdea': 'AI가 자동으로 설계 문서를 생성하여 개발 과정을 단순화',
  'currentProblem': '초보 개발자들이 복잡한 설계 과정을 이해하고 진행하는 데 어려움이 있음',
  'expectedBenefits': ['빠른 개발 착수', '효율적인 팀원 간 커뮤니케이션', '체계적인 프로젝트 경험']},
 'target_users': ['프로젝트 경험이 적은 초보 개발자'],
 'technology_stack': ['react',
  'typescript',
  'springboot',
  'AWS',
  'git action',
  'vercel'],
 'title': '프로젝트 관리 웹 서비스',
 'workspace_id': 1}

In [ ]:
# Jinja2 template 객체 생성
template = Template(RCMD_PROMPT)

# Rendering
rendered = template.render(input=json.dumps(project_summary, ensure_ascii=False, indent=2))
print(rendered)

#============================
#
# prompt = PromptTemplate(
#     input_variables=["input", "tools", "tool_names", "agent_scratched"],
#     template=RCMD_PROMPT
# )

당신은 신입/초보 개발자 팀을 이끄는 시니어 테크 리드입니다.  
다음은 한 프로젝트에 대한 주요 정보입니다.  
이 정보를 바탕으로 해당 프로젝트를 성공적으로 구현하기 위한 작업 구조를 작성해야 합니다.

---

- 프로젝트 정보 JSON:
{
  "category": "웹 서비스",
  "core_features": [
    "ERD 자동생성",
    "API 명세서 자동생성",
    "프로젝트 정보 자동생성"
  ],
  "problem_solving": {
    "solutionIdea": "AI가 자동으로 설계 문서를 생성하여 개발 과정을 단순화",
    "currentProblem": "초보 개발자들이 복잡한 설계 과정을 이해하고 진행하는 데 어려움이 있음",
    "expectedBenefits": [
      "빠른 개발 착수",
      "효율적인 팀원 간 커뮤니케이션",
      "체계적인 프로젝트 경험"
    ]
  },
  "target_users": [
    "프로젝트 경험이 적은 초보 개발자"
  ],
  "technology_stack": [
    "react",
    "typescript",
    "springboot",
    "AWS",
    "git action",
    "vercel"
  ],
  "title": "프로젝트 관리 웹 서비스",
  "workspace_id": 1
}

작업 조건:
1. Category: 패키지 수준의 상위 기능 그룹, 5개 이상  
2. Feature: 각 Category 아래 기능 모듈, 5개 이상  
3. Action: 각 Feature에 대응하는 실제 구현 단위, 3개 이상  
4. importance: 각 Actions의 기능 중요도 (1~5 사이 정수)  
5. 출력은 반드시 JSON 형식의 문자열만 반환 (설명 X, 주석 X)

예시 포맷:
{
  "workspace_id": "워크스페이스 ID",
  "recommendedCategories": [
 

In [ ]:
# OpenAI 객체 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
    max_tokens=1024,
    openai_api_key=openai_api_key
)

In [49]:
# # langchain template 생성
# langchain_prompt = PromptTemplate(
#     input_variables=["input", "tools", "tool_names", "agent_scratched"],
#     template=rendered  # jinja2 template & langchain prompt 충돌 막으려면 여기서 렌더링된 템플릿 전달
# )

# # llm(OpenAI) <-> Langchain 연결
# llm_chain = LLMChain(prompt=langchain_prompt, llm=llm)

#=====================

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [50]:
# LLM으로 category, feature, action 추천받는 함수
def generate_task_suggestion(input_dict: dict):
    return llm_chain.run(input_dict)

# project summary를 받아 추천 작업을 생성하는 메인 함수
def search_with_summary(input_summary: dict) -> str:

    rendered_prompt = input_summary  # 입력 데이터 보호

    # features = rendered_prompt.get("core_features", [])
    solution = rendered_prompt.get("problem_solving", {}).get("solutionIdea", "")
    tech_stack = rendered_prompt.get("technology_stack", [])

    joined_stack = " ".join(tech_stack)
    # truncated_solution = solution[:30]

    search_results = []

    # 웹 검색 툴 정의
    # search = DuckDuckGoSearchRun()
    search = SerpAPIWrapper()

    # SerpAPI 사용
    result = None
    try:
        query = f'{joined_stack} 기반 {solution} 구현 방법'
        result = search.run(query)
        # time.sleep(2)
        result_snippet = result[:500]
        print(result_snippet)  # test, type=str
    except Exception as e:
        print(f"[Search Error] SerpAPI 검색 실패: {e}")

    # LLM 사용
    generated_task = None
    try:
        # print(result_snippet)
        # llm_input = json.load(result_snippet)

        llm_input = {
            "input": result_snippet,
            # "workspace_id": input_summary.get("workspace_id", ""),
            # "tools": tool_descriptions,
            # "tool_names": ", ".join([t.name for t in tools]),
            # "agent_scratchpad": ""
        }
        generated_task = generate_task_suggestion(llm_input)
    except Exception as e:
        print(f"[LLM Error] 작업 생성 실패: {e}")

    search_results.append(generated_task)

    return generated_task

In [51]:
# test
project_id = int(random.choice(proj_id_list))
project_summary = extract_project_summary(data, project_id)

print(project_id)
project_summary

4


{'category': '웹사이트',
 'core_features': ['챗봇을 통한 대화 기반 추억 정보 수집',
  '사진 업로드 및 기억 저장',
  '디지털 다이어리 작성',
  '랜덤 추억 회상 기능'],
 'problem_solving': {'solutionIdea': '사용자가 사진을 업로드하고 챗봇과 대화하여 기억을 회상하고 정리하는 플랫폼 제공',
  'currentProblem': '치매 환자와 가족 간의 기억 회복 및 소통 부족',
  'expectedBenefits': ['치매 환자의 삶의 질 향상', '가족 간의 정서적 유대감 증진']},
 'target_users': ['손쉽게 추억을 기록하고 싶은 사람', '치매 환자와 그 가족'],
 'technology_stack': ['react', 'typescript', '컴퓨터 비전', 'springboot'],
 'title': '기억박물관',
 'workspace_id': 7}

In [52]:
search_with_summary(project_summary)  # test

['- 대체텍스트 : 시각장애 학습자가 사전에 강의 자료를 업로드하면, 그 자료에 포함된 그래프, 그림, 사진을 설명하는 대체텍스트를 생성한다. ... - 청각 ...', "... 제공' 서비스를 제공하고자 합니다. 사용자가 문제가 있는 망고잎의 사진을 앱에 업로드하면 이미지 객체 검출 및 분류 AI 기술을 활용하여 망고 질병을 식별해 줍니다."]
[LLM Error] 작업 생성 실패: Missing some input keys: {' tool_names ', ' tools ', ' agent_scratchpad ', ' input ', '\n  "workspace_id"'}


C:\Users\user\AppData\Local\Temp\ipykernel_6800\492503583.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm_chain.run(input_dict)


In [ ]:
# 툴 정의
# category, feature, action 초기 생성
create_tool = Tool(
    name="CreateWorkflow",
    func=search_with_summary,
    description="JSON 작업 흐름에 대해 category와 feature 각 5개 이상, actions 3개 이상씩 추천합니다."
)

# 출력용
final_tool = Tool(
    name="FinalAnswer",
    func=lambda x: x,
    description="최종 JSON 결과를 반환합니다.",
    return_direct=True  # 도구 호출 즉시 결과 반환, agent 종료 
)

tools = [create_tool, final_tool]
tool_names = [tool.name for tool in tools]

In [ ]:
agent = create_react_agent(
    llm=llm,
    tools=tools,  # 단순 tools 선언, AgentExecutor에서 tools 재전달 필요
    # prompt=langchain_prompt
    prompt=prompt
)

# 메모리 설정
memory = ConversationBufferMemory(
    memory_key="agent_scratchpad",   # 대화 로그 저장용
    input_key="input"                # 사람이 말한 내용이 들어가는 키
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,  # tools의 기능 전달
    memory=memory,
    max_iterations=3,
    max_execution_time=2,
    early_stopping_method="force",
    handle_parsing_errors=True,
    verbose=True
)

ValueError: Prompt missing required variables: {'agent_scratchpad', 'tools', 'tool_names'}

In [ ]:
response = agent_executor.invoke({
    "input": json.dumps(project_summary, ensure_ascii=False),
    "tool": ["CreateWorkflow", "Final Answer"],
    "tool_names": tool_names,
    "agent_scratchpad": ""
})

# 답변 생성 확인
print(response)

NameError: name 'agent_executor' is not defined